# Flag Additions Optimize

Optimizes file removing escape characters and decreasing the flag size

In [1]:
import requests
import json
import urllib

## Open the file

In [5]:
flagsPath = "./in/flags_additions.json"

with open(flagsPath, "r", encoding="utf-8") as json_file:
    flags = json.load(json_file)

## Unescape characters in flags links

In [22]:

for country in flags:
    regions = flags[country]['regions']
    for region in regions.values():
        flag = region['flag']
        region['flag'] = urllib.parse.unquote(flag)


## Reduce the flag size

### Undo previous png optimization

In [7]:
for country in flags.values():
    for flagData in country["regions"].values():
        flag: str = flagData["flag"]
        if "upload.wikimedia.org" not in flag:
            continue

        textToMatch = ".svg"
        constainsSvg = flag.find(textToMatch)
        if constainsSvg != -1:
            flagData["flag"] = flag[:constainsSvg + len(textToMatch)]


https://upload.wikimedia.org/wikipedia/commons/6/6a/Bandera_de_la_Provincia_de_Salta.svg
https://upload.wikimedia.org/wikipedia/commons/1/15/Bandera_de_la_Provincia_de_Buenos_Aires.svg
https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Bandera_de_la_Provincia_de_San_Luis.svg
https://upload.wikimedia.org/wikipedia/commons/6/60/Bandera_de_la_Provincia_de_La_Rioja.svg
https://upload.wikimedia.org/wikipedia/commons/0/07/Bandera_de_la_Provincia_de_Santiago_del_Estero.svg
https://upload.wikimedia.org/wikipedia/commons/3/33/Bandera_de_la_Provincia_del_Chaco.svg
https://es.wikipedia.org/wiki/Archivo:Flag_of_the_San_Juan_Province.svg
https://upload.wikimedia.org/wikipedia/commons/7/7b/Bandera_de_la_Provincia_de_Catamarca.svg
https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Bandera_de_la_Provincia_de_La_Pampa.svg
https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Bandera_de_la_Provincia_de_Mendoza.svg
https://upload.wikimedia.org/wikipedia/commons/c/ce/Bandera_de_la_Provin

In [23]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/4/43/Flag_of_Southwest_Papua.svg"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}


def urlContentSize(url):
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        file_size_bytes = len(response.content)
        return file_size_bytes / 1024
    else:
        print(
            f"Failed to fetch the image. Status code: {response.status_code}")


In [25]:
maxSvgSize = 100

for country in flags.values():
    for flagData in country["regions"].values():
        flag: str = flagData["flag"]
        extension = flag.split(".")[-1]
        if extension != "svg" or "upload.wikimedia.org" not in flag:
            continue
        size = urlContentSize(flag)
        if size > maxSvgSize:
            print("Flag:", flag, "Size:", size, "KB")

            sections = flag.split("/")
            commons_index = sections.index('commons')
            sections.insert(commons_index + 1, 'thumb')

            filename = sections[-1]
            addedFilename = "/128px-" + filename + ".png"
            sections.append(addedFilename)

            newFlag = '/'.join(sections[:])
            # print("New flag:", newFlag)
            # print("New flag:", newFlag, "Size:", urlContentSize(newFlag), "KB")
            flagData["flag"] = newFlag


Flag: https://upload.wikimedia.org/wikipedia/commons/6/65/Flag_of_Cundinamarca.svg Size: 141.3740234375 KB
New flag: https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/Flag_of_Cundinamarca.svg//128px-Flag_of_Cundinamarca.svg.png
Flag: https://upload.wikimedia.org/wikipedia/commons/9/9b/Flag_of_Guaviare.svg Size: 231.962890625 KB
New flag: https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Flag_of_Guaviare.svg//128px-Flag_of_Guaviare.svg.png
Flag: https://upload.wikimedia.org/wikipedia/commons/3/39/Flag_of_Sucre_(Colombia).svg Size: 286.935546875 KB
New flag: https://upload.wikimedia.org/wikipedia/commons/thumb/3/39/Flag_of_Sucre_(Colombia).svg//128px-Flag_of_Sucre_(Colombia).svg.png
Flag: https://upload.wikimedia.org/wikipedia/commons/f/f7/Flag_of_South_Bohemian_Region.svg Size: 165.423828125 KB
New flag: https://upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Flag_of_South_Bohemian_Region.svg//128px-Flag_of_South_Bohemian_Region.svg.png
Flag: https://upload.wikimedia.o

In [26]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()

with open(flagsPath, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", flagsPath)


JSON data has been saved to ./in/flags_additions.json
